In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import helper
from importlib import reload
%config InlineBackend.figure_format='retina'

In [5]:
reload(helper)

<module 'helper' from '/Users/beili/Dropbox/Competitions/Kaggle/New_House_Prices/helper.py'>

In [3]:
data = helper.load_data('ameshousing.txt', 'Order')
data.head()

PID  MS SubClass MS Zoning  Lot Frontage  Lot Area Street Alley  \
Order                                                                          
1      526301100           20        RL         141.0     31770   Pave   NaN   
2      526350040           20        RH          80.0     11622   Pave   NaN   
3      526351010           20        RL          81.0     14267   Pave   NaN   
4      526353030           20        RL          93.0     11160   Pave   NaN   
5      527105010           60        RL          74.0     13830   Pave   NaN   

      Lot Shape Land Contour Utilities    ...     Pool Area Pool QC  Fence  \
Order                                     ...                                
1           IR1          Lvl    AllPub    ...             0     NaN    NaN   
2           Reg          Lvl    AllPub    ...             0     NaN  MnPrv   
3           IR1          Lvl    AllPub    ...             0     NaN    NaN   
4           Reg          Lvl    AllPub    ...             0     NaN    NaN   
5           IR1          Lvl    AllPub    ...             0     NaN  MnPrv   

      Misc Feature Misc Val Mo Sold Yr Sold  Sale Type  Sale Condition  \
Order                                                                    
1              NaN        0       5    2010        WD           Normal   
2              NaN        0       6    2010        WD           Normal   
3             Gar2    12500       6    2010        WD           Normal   
4              NaN        0       4    2010        WD           Normal   
5              NaN        0       3    2010        WD           Normal   

       SalePrice  
Order             
1         215000  
2         105000  
3         172000  
4         244000  
5         189900  

[5 rows x 81 columns]

In [ ]:
# This section studies each data feature independently. Characteristics of 
# the data to be examined include class counts for categorical variables, 
# descriptive statistics on continuous variables, data shape, data types, and the number
# of missing or null values.

data_shape = data.shape
print('The data shape is {}'.format(data_shape))

data_types = data.info()
print(data_types)

In [6]:
# Create a list of the categorical variables in the data
# Note that features involving year is not considered categorical for this analysis,
# since the difference between the years of two records may affect the target variable

catg_vars = helper.get_catg_vars(data, 'PID', 'MS SubClass', 'Overall Qual', 'Overall Cond')
print(catg_vars.columns)
print('\nThere are {} categorical variables'.format(len(catg_vars.columns)))

Index(['MS Zoning', 'Street', 'Alley', 'Lot Shape', 'Land Contour',
       'Utilities', 'Lot Config', 'Land Slope', 'Neighborhood', 'Condition 1',
       'Condition 2', 'Bldg Type', 'House Style', 'Roof Style', 'Roof Matl',
       'Exterior 1st', 'Exterior 2nd', 'Mas Vnr Type', 'Exter Qual',
       'Exter Cond', 'Foundation', 'Bsmt Qual', 'Bsmt Cond', 'Bsmt Exposure',
       'BsmtFin Type 1', 'BsmtFin Type 2', 'Heating', 'Heating QC',
       'Central Air', 'Electrical', 'Kitchen Qual', 'Functional',
       'Fireplace Qu', 'Garage Type', 'Garage Finish', 'Garage Qual',
       'Garage Cond', 'Paved Drive', 'Pool QC', 'Fence', 'Misc Feature',
       'Sale Type', 'Sale Condition', 'PID', 'MS SubClass', 'Overall Qual',
       'Overall Cond'],
      dtype='object')

There are 47 categorical variables


In [ ]:
def conv_to_catg_type(data, catg_vars):
    '''
        Convert the categorical values to data type category.
    '''
    data[catg_vars] = data[catg_vars].apply(lambda x: x.astype('category'))
    
    return data

data = conv_to_catg_type(data, catg_vars)
data.info()

In [ ]:
## Next, let's deal with the null values in the dataset
def get_null_vars(data):
    '''
        Returns the column names in a dataframe of the columns with null values.
        The output is separated by categorical and quantitative variables.
    '''
    catg_null_vars = []
    quant_null_vars = []
    vars_w_na_data = data.columns[data.isnull().any()].tolist()
    for var in vars_w_na_data:
        if ((data[var].dtype.name == 'category') or (data[var].dtype.name == 'object')):
            catg_null_vars += [var]
        else:
            quant_null_vars += [var]
    
    return catg_null_vars, quant_null_vars

In [ ]:
catg_null_vars, quant_null_vars = get_null_vars(data)
print(catg_null_vars)
print(quant_null_vars)

In [ ]:
def get_null_ix(data, col_names):
    '''
        Given a list of column names, return the indices of the rows where the null values
        of those columns are located.
    '''
    
    ix_dict = {col: data[data[col].isnull()].index for col in col_names}
    
    return ix_dict
    

In [ ]:
pd.set_option('display.max_columns', None)
pd.set_option('precision', 2)

In [ ]:
# Investigate the numeric variables
data_num = get_num_vars(data)

In [ ]:
# Start with univariate analysis
print(data_num.columns.values)
print('\n')
print(data_num.describe())

In [ ]:
# Check the data skews
skews = data_num.skew()
print(skews.sort_values(ascending=False))

In [ ]:
# Plot histograms of the numeric variables
data_num.hist(figsize=(10, 20), layout=(-1, 4))
plt.tight_layout()
plt.show()


In [ ]:
# The skew calculations and the histograms show 
# that some variable have high skew
# Examine a boxplot for Lot Area, which is right-skewed
fig, ax = plt.subplots()
ax.boxplot(data_num['Lot Area'])
ax.set_xlabel('Lot Area Sq Ft.')
plt.show()

In [ ]:
# Apply a log transform to the lot area variable
data_num['Log Lot Area'] = np.log10(data_num['Lot Area'] + 1)

In [ ]:
# Plot the histogram of the log transform
fig, ax = plt.subplots()
ax.hist(data_num['Log Lot Area'])
ax.set_xlabel('Log10 of Lot Area')
plt.show()

In [ ]:
# The histograms review that some features such as Bsmt Unf SF follow an exponential distribution
# Apply a box-cox transform to such features so that their distribution can be more Gaussian
from scipy.stats import boxcox
open_porch_boxcox, lda = boxcox(data_num['Open Porch SF'].values + 1)

In [ ]:
plt.hist(open_porch_boxcox)
plt.show()

In [ ]:
bsmt_unf_log = np.log(data_num['Bsmt Unf SF'] + 1)

In [ ]:
bsmt_unf = data_num['Bsmt Unf SF'].dropna()

In [ ]:
bsmt_unf_boxcox, lda = boxcox(bsmt_unf + 1)

In [ ]:
print(bsmt_unf_boxcox)
print(lda)

In [ ]:
plt.hist(bsmt_unf_boxcox)
plt.show

In [ ]:
# Check the correlations between the numeric variables
# corr = data_num.corr()
fig = plt.figure(figsize=(20, 20))
ax = fig.add_subplot(111)
cax = ax.matshow(corr, vmin=-1, vmax=1)
fig.colorbar(cax)
names = data_num.columns.values
ticks = np.arange(0, len(names), 1)
ax.set_xticks(ticks)
ax.set_yticks(ticks)
_ = ax.set_xticklabels(names)
_ = ax.set_yticklabels(names)
_ = plt.xticks(rotation=-90)

In [ ]:
null_ix = get_null_ix(data, catg_null_vars)

In [ ]:
pd.set_option('display.max_columns', None)

In [ ]:
data.loc[null_ix['Electrical']]

In [ ]:
def fill_nulls(data, **kwargs):
    '''
        Given a set of keyword arguments, fill in the null values of a dataframe column, where the
        key is the column name and the value is the fill value.
    '''
    
    for col, value in kwargs.items():
        try:
            data[col].cat.add_categories([value], inplace=True)
        except:
            pass
        data[col].fillna(value=value, inplace=True)
        

In [ ]:
fill_dict = {'Alley': 'No Alley Access', 'Mas Vnr Type': 'None', 'Bsmt Qual': 'No Basement', 
             'Bsmt Cond': 'No Basement', 'BsmtFin Type 1': 'No Basement', 'BsmtFinType 2': 'No Basement', 
             'Eletrical': ''}
fill_nulls(data, **fill_dict)

In [ ]:
# From the codebook, a Null value for alley means no alley access so we can fill that in as a separate class
try:
    data['Alley'].cat.add_categories(['No Alley Access'], inplace=True)
except:
    pass
data['Alley'].fillna(value='No Alley Access', inplace=True)

In [ ]:
# From the codebook, a Null value for Mas Vnr Type should be None
data['Mas Vnr Type'].fillna(value='None', inplace=True)

In [ ]:
# From the codebook, a Null value for Bsmt Qual indicates No Basement
try:
    data['Bsmt Qual'].cat.add_categories(['No Basement'], inplace=True)
except:
    pass
data['Bsmt Qual'].fillna(value='No Basement', inplace=True)


In [ ]:
# Make the basement qual variable an ordinal variable
catg_order = ['No Basement', 'Po', 'Fa', 'TA', 'Gd', 'Ex']
data['Bsmt Qual'] = pd.Categorical(data['Bsmt Qual'], ordered=True, categories=catg_order)

In [ ]:
# From the codebook, a Null value for Bsmt Cond indicates No Basement
try:
    data['Bsmt Cond'].cat.add_categories(['No Basement'], inplace=True)
except:
    pass
data['Bsmt Cond'].fillna(value='No Basement', inplace=True)

# Make the basement cond variable an ordinal variable
catg_order = ['No Basement', 'Po', 'Fa', 'TA', 'Gd', 'Ex']
data['Bsmt Cond'] = pd.Categorical(data['Bsmt Cond'], ordered=True, categories=catg_order)

## Note: There are equal number of null values for Bsmt Qual as for Bsmt Cond, which makes sense

In [ ]:
# Define a helper function to show the distribution of records across all labels for a
# given set of categorical variables
def show_label_distr(df, catg_vars):
    for var in catg_vars:
        # Replace null values with a label
        if any(pd.isnull(data[var])):
            df[var].fillna('None', inplace=True)
        print(df.groupby(var).size().sort_values(ascending=False), '\n')

In [ ]:
show_label_distr(data, vars_w_na_data)

In [ ]:
# Get the class counts for categorical variables
ms_subclass = data.groupby('MS SubClass').size().sort_values(ascending=False)
print(ms_subclass)

In [ ]:
ms_zoning = data.groupby('MS Zoning').size().sort_values(ascending=False)
print(ms_zoning)

In [ ]:
street = data.groupby('Street').size().sort_values(ascending=False)
print(street)

In [ ]:
alley = data.groupby('Alley').size()
print(alley)
print('There are {} null values in Alley'.format(len(data) - alley.sum()))

In [ ]:
# Fill in alley null values with another meaningful label
data['Alley'].fillna('No Alley Access'.astype('category'), inplace=True)
alley = data.groupby('Alley').size()
print(alley)

In [ ]:
data.columns